# Research Summary Template

Use this notebook as a starting point for digest-agent exploration.

1. Load sanitized research notes from `.workspace/metrics/`.
2. Issue a signed gateway request with `ResearchBridge.research_query`.
3. Persist notebook outputs to `storage/analytics/pipelines/` for audit trails.


In [ ]:
from pathlib import Path
import pandas as pd
from notebooks.lib.research_bridge import ResearchBridge, GatewayRequestError

bridge = ResearchBridge()
sample_path = bridge.sample_dataset_path("research_notes_sample.csv")
notes_df = pd.read_csv(sample_path)
display(notes_df.head())

try:
    query_response = bridge.research_query(
        query="Summarise sanitized research notes",
        data_sources=["research_notes_sample.csv"],
        analysis_type="literature-review",
    )
except GatewayRequestError as exc:
    print(f"Gateway offline or unreachable: {exc}")
    query_response = {"message": "offline demonstration"}

summary_payload = {
    "headline": query_response.get("message", "no message returned"),
    "projects": notes_df["project_id"].tolist(),
    "note_count": int(notes_df.shape[0]),
}

artifact_path = bridge.push_summary(
    pipeline_name="digest-agent-metrics",
    summary=summary_payload,
    metadata={"template": "research_summary", "dataset": "research_notes_sample.csv"},
)
print(f"Summary stored at: {artifact_path}")
